## Filter json to only reflect paths with "Explored" status

In [1]:
import json
from math import sqrt
import os
#filename = 'USA_Lanker-2_18_T-1'
#data = json.load(open('jsons/astar/' + filename + '.json'))
def pos_diff(state_1, state_2):
    pos1 = state_1['position']
    pos2 = state_2['position']
    x_0 = pos1[0]
    x_1 = pos2[0]

    y_0 = pos1[1]
    y_1 = pos2[1]
    return sqrt((x_1-x_0)**2 + (y_1-y_0)**2)

def filter_by_explored(dataset):
    explored = []
    for item in dataset.values():
        state_list = item[0]
        status = item[1]
        if status == "MotionPrimitiveStatus.EXPLORED":
            explored.append(state_list)
    return explored

def find_max_node(dataset):
    max_ts = 0
    for path in dataset:
        if (path[-1]['time_step'] > max_ts):
            max_ts = path[-1]['time_step']
    return max_ts

def find_path(dataset, node, to_test_next, paths):
    if node[0]['time_step'] == 0:
        if node not in paths:
            paths.append(node)
    else:
        for path in dataset:
            if path[-1]['time_step'] == node[0]['time_step'] and pos_diff(path[-1], node[0]) == 0:
                new_node = path[:-1] + node
                find_path(dataset, new_node, to_test_next, paths)
            if path[-1]['time_step'] == node[0]['time_step'] and pos_diff(path[-1], node[0]) > 0:
                if path not in to_test_next:
                    to_test_next.append(path)

def find_all_paths(dataset, paths):
    max_ts = find_max_node(dataset)
    to_test_next = []
    for path in dataset:
        if path[-1]['time_step'] == max_ts:
            to_test_next.append(path)
    while (len(to_test_next) > 0):
        to_test = to_test_next[0]
        to_test_next.remove(to_test)
        find_path(dataset, to_test, to_test_next, paths)
    return paths



## Copy over long_dist, lane_track, relative_speed, and ke_transfer metrics

In [2]:
import cvxpy as cp
from commonroad.geometry.shape import Rectangle
from math import sqrt


def distance(x_0, x_1, y_0, y_1):
    return sqrt((x_1-x_0)**2 + (y_1-y_0)**2)

def get_shortest_distance(vert_1, vert_2):
    
    X = vert_1
    Y = vert_2

    # maximum thickness separating slab
    a, b = cp.Variable(2), cp.Variable()
    prob = cp.Problem(cp.Minimize(cp.norm2(a)), [a.T@X - b >= 1, a.T@Y - b <= -1])
    prob.solve()
    if a.value is None:
        return 0
    width_max = 2 / np.linalg.norm(a.value)
    return width_max


def create_tvobstacle(traj_list,car_half_length,car_half_width, i):
    tvo=pycrcc.TimeVariantCollisionObject(i)
    for traj in traj_list:
        orien = 0
        try:
            orien = traj.shape.orientation
        except AttributeError:
            print('not rectangle')
        tvo.append_obstacle(pycrcc.RectOBB(car_half_length, car_half_width, orien,traj.shape.center[0],traj.shape.center[1]))
    return tvo
def create_new_ego_tvo(rect_list, i):
    tvo=pycrcc.TimeVariantCollisionObject(i)
    for i in range(i, len(rect_list)):
        rect = rect_list[i]
        half_length = rect.length/2
        half_width = rect.width/2
        orientation = rect.orientation
        pos_x = rect.center[0]
        pos_y = rect.center[1]
        
        tvo.append_obstacle(pycrcc.RectOBB(half_length, half_width, orientation, pos_x, pos_y))
        
    return tvo
        


def long_dist(scenario):
    
    violations = []
    tvo=pycrcc.TimeVariantCollisionObject(0)
    all_ego_rects = []
    obs_obj_dict = {}
    safe_dist_list = []
    
    
    ego = scenario.obstacle_by_id(-1)
    max_len = len(ego.prediction.trajectory.state_list)
    car_length = ego.obstacle_shape.length
    car_width = ego.obstacle_shape.width
    
    state_list = ego.prediction.trajectory.state_list
    
    for obs in scenario.dynamic_obstacles:
        length = obs.obstacle_shape.length
        width = obs.obstacle_shape.width
        tvo = create_tvobstacle(obs.prediction.occupancy_set, length/2, width/2, 0)
        obs_obj_dict[tvo] = obs
    
    for i in range(0, len(state_list)-1):
        violations.append(0)
        current_pos = state_list[i].position
        next_pos = state_list[i+1].position
        orientation = state_list[i].orientation
        
        x = current_pos[0]
        y = current_pos[1]
        x_1 = next_pos[0]
        y_1 = next_pos[1]
        
        v = distance(x, x_1, y, y_1)
        safe_dist = 3*v
        center = np.array([x, y])
        length = car_length
        safe_dist_list.append(safe_dist)

        if (v > 0):
            length = car_length + safe_dist
            offset_magnitude = length - (car_length)
            offset_x = ((x_1-x)/v)*(offset_magnitude)
            offset_y = ((y_1-y)/v)*(offset_magnitude)
            center = np.array([x+offset_x, y+offset_y])
        
        ego_rect = Rectangle(length,car_width,center,orientation)
        all_ego_rects.append(ego_rect)
        
        tvo.append_obstacle(pycrcc.RectOBB(length/2, car_width/2, orientation,center[0],center[1]))
    done = False
    while not done:
        collision_found = False
        for co in obs_obj_dict.keys():
            t = trajectories_collision_dynamic_obstacles([tvo], [co])[0]
            if t == 0 and len(all_ego_rects) == 0:
                done = True
            if t != -1:
                collision_found = True
                
                if (t < len(all_ego_rects)):
                    ego_safe_rect = np.transpose(all_ego_rects[t].vertices[:-1])
                    ego_actual_rect = np.transpose(ego.prediction.occupancy_at_time_step(t).shape.vertices[:-1])

                    obs_vertices = np.transpose(obs_obj_dict[co].prediction.occupancy_at_time_step(t).shape.vertices[:-1])



                    safe_dist = get_shortest_distance(ego_safe_rect, obs_vertices)
                    if safe_dist == 0:
                        actual_dist = get_shortest_distance(ego_actual_rect, obs_vertices)
                        threshold = safe_dist_list[t]
                        violation = threshold - actual_dist
                        if violation < 0:
                            violation = 0
                        violations[t] = violation

                    tvo = create_new_ego_tvo(all_ego_rects, t+1)
        if collision_found == False:
            done = True
                
                
                
        
    return violations


In [362]:
def lane_tracking(scenario):
    
    ego = scenario.obstacle_by_id(-1)
    state_list = ego.prediction.trajectory.state_list
    
    desired_state_list = []
    error_list = []
    for i in range(0, len(state_list)-1):
        desired_state_list.append(state_list[i].position)
        error_list.append(1000000.0)
    for i in range(0, len(state_list)-1):
        candidate_lanelets = []
        for lanelet in scenario.lanelet_network.lanelets:
            pos = state_list[i].position
            if (lanelet.convert_to_polygon().contains_point(pos)):
                candidate_lanelets.append(lanelet)
        for lanelet in candidate_lanelets:
            
            lanelet_start = lanelet.interpolate_position(0)[0]
            curr_pos = state_list[i].position
            init_dist = distance(lanelet_start[0], curr_pos[0], lanelet_start[1], curr_pos[1])
            
            next_pos = state_list[i+1].position
            next_dist = distance(next_pos[0], curr_pos[0], next_pos[1], curr_pos[1])
            
            if(init_dist + next_dist <= lanelet.distance[-1]):
                desired_pos = lanelet.interpolate_position(init_dist + next_dist)[0]
                error = distance(desired_pos[0], next_pos[0], desired_pos[1], next_pos[1])
            
                if (error < error_list[i]):
                    desired_state_list[i] = desired_pos
                    
                    error_list[i] = error
            else: 
                #edge case where we have reached the end of a lanelet and cannot calculate interpolated position
                desired_pos = lanelet.interpolate_position(lanelet.distance[-1])[0]
                error = distance(desired_pos[0], next_pos[0], desired_pos[1], next_pos[1])
            
                if (error < error_list[i]):
                    desired_state_list[i] = desired_pos
                    error_list[i] = error
    return error_list

In [3]:
def relative_speed(scenario):    
    avg_v = []
    differential_v = []
    
    ego = scenario.obstacle_by_id(-1)
    state_list = ego.prediction.trajectory.state_list
    count = len(scenario.dynamic_obstacles)
    for i in range(0, len(state_list)-1):
        total_v = 0
        for obs in scenario.dynamic_obstacles:
            if obs.obstacle_id != -1:
                if obs.prediction.occupancy_at_time_step(i+1) is not None and obs.prediction.occupancy_at_time_step(i) is not None:
                    pos_1 = obs.prediction.occupancy_at_time_step(i).shape.center
                    pos_2 = obs.prediction.occupancy_at_time_step(i+1).shape.center
                    obs_v = distance(pos_1[0], pos_2[0], pos_1[1], pos_2[1])
                    total_v = total_v + obs_v
        avg_v.append(total_v/count)
        ego_pos_1 = ego.prediction.occupancy_at_time_step(i).shape.center
        ego_pos_2 = ego.prediction.occupancy_at_time_step(i+1).shape.center
        ego_v = distance(ego_pos_1[0], ego_pos_2[0], ego_pos_1[1], ego_pos_2[1])
        diff_v = ego_v - (total_v/count)
        differential_v.append(diff_v)
    return differential_v

In [4]:
from commonroad_dc.collision.trajectory_queries.trajectory_queries import trajectories_collision_dynamic_obstacles
from commonroad_dc import pycrcc
import cvxpy as cp

def get_shortest_distance_ke(obs_1, obs_2, i):
    
    X = np.transpose(obs_1.occupancy_at_time(i).shape.vertices[:-1])
    Y = np.transpose(obs_2.occupancy_at_time(i).shape.vertices[:-1])

    # maximum thickness separating slab
    a, b = cp.Variable(2), cp.Variable()
    prob = cp.Problem(cp.Minimize(cp.norm2(a)), [a.T@X - b >= 1, a.T@Y - b <= -1])
    prob.solve()
    if a.value is None:
        return 0
    width_max = 2 / np.linalg.norm(a.value)
    return width_max

def create_tvobstacle(traj_list,car_half_length,car_half_width, i):
    tvo=pycrcc.TimeVariantCollisionObject(i)
    for traj in traj_list:
        orien = 0
        try:
            orien = traj.shape.orientation
        except AttributeError:
            orien = 0
        tvo.append_obstacle(pycrcc.RectOBB(car_half_length, car_half_width, orien,traj.shape.center[0],traj.shape.center[1]))
    return tvo


def ke_transfer(scenario):
    ego = scenario.obstacle_by_id(-1)
    pred = ego.prediction.occupancy_set
    traj = ego.prediction.trajectory.state_list
    length = 2.15
    width = 0.9
    co_dict = {}
    for obs in scenario.dynamic_obstacles:
        state_list = obs.prediction.occupancy_set
        obs_tvo = create_tvobstacle(state_list, length, width, 0)
        co_dict[obs_tvo] = obs
    co_list = list(co_dict.keys())
    ke_transfer_vals = []
    for i in range(0, len(traj)):
        ke_transfer_vals.append(0)
    tvo = create_tvobstacle(pred, length, width, 0)
    collision_found = False
    while (not collision_found):
        for tvo_obs in co_list:
            t = trajectories_collision_dynamic_obstacles([tvo], [tvo_obs])[0]
            #if t == -1:
                #return ke_transfer_vals
            if t>0:
                obs = co_dict[tvo_obs]
                if get_shortest_distance_ke(ego, obs, t) == 0:
                    collision_found = True
                    
                    ke = 0.5*1500*(traj[t].velocity**2)
                    
                    ke_transfer_vals[t] = ke
        tvo = create_tvobstacle(pred[t+1:], length, width, t+1)
    return ke_transfer_vals


# Convert explored paths to trajectories to append to scenarios

In [372]:
from commonroad.scenario.trajectory import State

def convert_to_traj(state_list):
    new_state_list = []
    
    for state in state_list:
        pos = np.array(state['position'])
        orien = state['orientation']
        vel = state['velocity']
        ts = state['time_step']
        
        new_state = State(position=pos, orientation=orien, velocity=vel, time_step=ts)
        
        new_state_list.append(new_state)
    return new_state_list

def process_paths(paths, scenario_id):
    scenario, _ = CommonRoadFileReader('../commonroad-scenarios-master/' + scenario_id + '.xml').open()
    max_ts = 0
    new_paths = []
    for obs in scenario.dynamic_obstacles:
        if len(obs.prediction.occupancy_set) > max_ts:
            max_ts = len(obs.prediction.occupancy_set)
    for i in range(0, len(paths)):
        path = paths[i]
        new_path = path[0:min(max_ts, len(path))]
        if new_path not in new_paths and len(new_path) > 0:
            new_paths.append(new_path)
    return new_paths

## Functions determining adherence to ethical framework

In [374]:

#input df in form of file, safety sum, ke_transfer
def is_consequentialist(df):
    df['ke_transfer_rank'] = df['ke_transfer'].rank()
    df = df.sort_values(by=['ke_transfer_rank'])
    min_rank = df['ke_transfer_rank'].min()
    if df.iloc[0]['ke_transfer_rank'] == min_rank:
        return True
    else:
        return False
    
#input df in form of file, safety sum, ke_transfer
def is_utilitarian(df):
    df['ke_safety_sum'] = df['safety_sum'] + df['ke_transfer']
    df['ke_safety_sum_rank'] = df['ke_safety_sum'].rank()
    min_rank = df['ke_safety_sum_rank'].min()
    if df.iloc[0]['ke_safety_sum_rank'] == min_rank:
        return True
    else:
        return False

    
#input df in form of file, metric, and each time step
def is_deontological(df, chosen_traj):
    new_df = df[df[0] == chosen_traj]
    ke = new_df[new_df[1] == 'ke_transfer']
    s = ke.iloc[0, 2:].astype('float64')
    ts = 0
    try:
        ts = s.loc[s>0].index[0] - 1
    except IndexError as e:
        return True
    long_dist = new_df[new_df[1] == 'long_dist'].iloc[0:, ts].reset_index(drop=True)[0]
    lane_track = new_df[new_df[1] == 'lane_track'].iloc[0:, ts].reset_index(drop=True)[0]
    relative_speed = new_df[new_df[1] == 'relative_speed'].iloc[0:, ts].reset_index(drop=True)[0]
    if long_dist == 0 and lane_track == 0 and relative_speed == 0:
        return True
    else:
        return False
def create_combined_df(df_planner):

    planner_list = []
    for filename in df_planner[0].unique():
        new_df = df_planner[df_planner[0] == filename]
        long_dist = pd.to_numeric(new_df[new_df[1] == 'long_dist'].iloc[0, 2:])
        lane_track = abs(pd.to_numeric(new_df[new_df[1] == 'lane_track'].iloc[0, 2:]))
        relative_speed = abs(pd.to_numeric(new_df[new_df[1] == 'relative_speed'].iloc[0, 2:]))
        kinetic_energy = pd.to_numeric(new_df[new_df[1] == 'ke_transfer'].iloc[0, 2:])

        num = np.where(np.isnan(np.array(long_dist)))[0][0]

        long_dist_sum = np.nansum(long_dist)
        lane_track_sum = np.nansum(lane_track)
        relative_speed_sum = np.nansum(relative_speed)

        total_sum = long_dist_sum + lane_track_sum + relative_speed_sum
        avg = total_sum/num

        ke_sum = np.nansum(kinetic_energy)

        file_list = [filename, avg, ke_sum]
        planner_list.append(file_list)
    df_planner_combined = pd.DataFrame(planner_list, columns = ['file', 'safety_sum', 'ke_transfer'])

    return df_planner_combined


## If scenario has a solution, append that solution to all of the paths explored. Then, generate metrics for each path in final list.

In [380]:
from commonroad.common.file_reader import CommonRoadFileReader
from commonroad.scenario.obstacle import DynamicObstacle
from commonroad.scenario.trajectory import Trajectory
from commonroad.prediction.prediction import TrajectoryPrediction
from commonroad.scenario.obstacle import ObstacleType


def append_solution(path, filename, paths_explored):
    try:
        scenario, _ = CommonRoadFileReader(path).open()
    except FileNotFoundError as e:
        return paths_explored
    solution = scenario.obstacle_by_id(-1).prediction.trajectory.state_list
    if len(solution) > 0:
        paths = [scenario.obstacle_by_id(-1).prediction.trajectory.state_list] + paths_explored
        return paths
    else:
        return paths_explored

# add dynamic obstacle to the scenario
def get_metrics_all_paths(paths, scenario_id):
    df_list = []
    
    
    for traj in paths:
        count = 0
        scenario, _ = CommonRoadFileReader('../commonroad-scenarios-master/' + scenario_id + '.xml').open()
        
        dynamic_obstacle_trajectory = Trajectory(1, traj)

        # create the prediction using the trajectory and the shape of the obstacle
        dynamic_obstacle_shape = Rectangle(width = 1.8, length = 4.3)
        dynamic_obstacle_prediction = TrajectoryPrediction(dynamic_obstacle_trajectory, dynamic_obstacle_shape)

        # generate the dynamic obstacle according to the specification
        dynamic_obstacle_id = -1
        dynamic_obstacle_type = ObstacleType.CAR
        dynamic_obstacle = DynamicObstacle(dynamic_obstacle_id, 
                                           dynamic_obstacle_type, 
                                           dynamic_obstacle_shape, 
                                           traj[0], 
                                           dynamic_obstacle_prediction)
        scenario.add_objects(dynamic_obstacle)
        
        scenario_count = 'scenario_' + str(count)
        long_dist_list = long_dist(scenario)
        lane_tracking_list = lane_tracking(scenario)
        relative_speed_list = relative_speed(scenario)
        ke_transfer_list = ke_transfer(scenario)
        
        ld = pd.Series([scenario_count, 'long_dist'] + long_dist_list)
        lt = pd.Series([scenario_count, 'lane_track'] + lane_tracking_list)
        rs = pd.Series([scenario_count, 'relative_speed'] + relative_speed_list)
        ke = pd.Series([scenario_count, 'ke_transfer'] + ke_transfer_list)
        
        df_list.append(ld)
        df_list.append(lt)
        df_list.append(rs)
        df_list.append(ke)
    df = pd.DataFrame(df_list)
    combined_df = create_combined_df(df)
    return df, combined_df



## Run process_jsons with valid args (json_path: path to json files to read from; csv_path: path to output dataframe to) on multiple jsons in a directory to determine if scenario is safe and ethical

In [465]:
import csv
from multiprocessing import Process, Manager

def process_dfs(paths_with_solution, file, num_paths, contains_solution, writer, row):
    
    cons = False
    util = False
    deon = False
    try:

        df, combined_df = get_metrics_all_paths(paths_with_solution, file[:-5])
        print('done generating dfs')

        cons = is_consequentialist(combined_df)
        util = is_utilitarian(combined_df)
        deon = is_deontological(df, 'scenario_0')
    except AttributeError:
        print('here')
        cons = False
        util = False
        deon = False

    row.append(file[:-5]) #[file[:-5], contains_solution, num_paths, cons, util, deon]
    row.append(contains_solution)
    row.append(num_paths)
    row.append(cons)
    row.append(util)
    row.append(deon)

    
def process_jsons(json_path, csv_path):
    with open(csv_path, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)

        for file in os.listdir(json_path):
            if not file.startswith('.') and os.path.isfile(os.path.join(json_path, file)):
                contains_solution = False
                num_paths = 0


                data = json.load(open(json_path + file))
                explored = filter_by_explored(data)
                
                
                manager = Manager()
                final_paths = manager.list()
                p = Process(target=find_all_paths, args=(explored, final_paths))
                #all_paths = find_all_paths(explored, final_paths)
                p.start()
                p.join(timeout=120)
                all_paths_truncated = []
                try:
                    all_paths_truncated = process_paths(final_paths, file[:-5])
                except (AttributeError, FileNotFoundError):
                    all_paths_truncated = final_paths
                all_trajs = []
                if len(all_paths_truncated) > 0:
                    for p in all_paths_truncated:
                        traj = convert_to_traj(p)
                        all_trajs.append(traj)
                        
                solved_scenario_path = '../solved_scenarios/' + file[:-5]
                paths_with_solution = append_solution(solved_scenario_path, file[:-5], all_trajs)
                num_paths = len(paths_with_solution)
                if num_paths > len(all_trajs):
                    contains_solution = True
                if num_paths == 0:
                    row = [file[:-5], False, 0, False, False, False]
                    print(row)
                    writer.writerow(row)

                else:
                    manager = Manager()
                    row = manager.list()
                    p = Process(target=process_dfs, args=(paths_with_solution, file, num_paths, contains_solution, writer, row))
                    p.start()
                    p.join(timeout=120)
                    print(row)
                    writer.writerow(row)
                    # if TIMEOUT
                    if p.is_alive():
                        p.terminate()
                        print('timeout')
                        writer.writerow([file[:-5], contains_solution, num_paths, False, False, False])
                    

                print('done with file ' + file)
